In [1]:
import random
import string

In [2]:
def getWordScore(word, n):
    """
    Returns the score for a word. Assumes the word is a valid word.

    The score for a word is the sum of the points for letters in the
    word, multiplied by the length of the word, PLUS 50 points if all n
    letters are used on the first turn.

    Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is
    worth 3, D is worth 2, E is worth 1, and so on (see SCRABBLE_LETTER_VALUES)

    word: string (lowercase letters)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    returns: int >= 0
    """
    points=0
    bonus=0
    total=0
    if len(word)==n:
        bonus=50
    else:
        bonus=0
    for letter in word:
        points=points+SCRABBLE_LETTER_VALUES[letter]
    total=points*len(word)+bonus
    return total


In [3]:
def dealHand(n):
    """
    Returns a random hand containing n lowercase letters.
    At least n/3 the letters in the hand should be VOWELS.

    Hands are represented as dictionaries. The keys are
    letters and the values are the number of times the
    particular letter is repeated in that hand.

    n: int >= 0
    returns: dictionary (string -> int)
    """
    hand={}
    numVowels = n // 3
    
    for i in range(numVowels):
        x = VOWELS[random.randrange(0,len(VOWELS))]
        hand[x] = hand.get(x, 0) + 1
        
    for i in range(numVowels, n):    
        x = CONSONANTS[random.randrange(0,len(CONSONANTS))]
        hand[x] = hand.get(x, 0) + 1
        
    return hand


In [4]:
def updateHand(hand, word):
    """
    Assumes that 'hand' has all the letters in word.
    In other words, this assumes that however many times
    a letter appears in 'word', 'hand' has at least as
    many of that letter in it. 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """
    n=hand.copy()
    for letter in word:
        n[letter]=n.get(letter,0)-1
    return n

In [5]:
def isValidWord(word, hand, wordList):
    """
    Returns True if word is in the wordList and is entirely
    composed of letters in the hand. Otherwise, returns False.

    Does not mutate hand or wordList.
   
    word: string
    hand: dictionary (string -> int)
    wordList: list of lowercase strings
    """
    count=0
    n=hand.copy()
    if word in wordList:
        for letter in word:
            if n.get(letter,0)!=0:
                count+=1
                n[letter]=n.get(letter,0)-1
        return count==len(word)
    else:
        return False


In [6]:
def calculateHandlen(hand):
    """ 
    Returns the length (number of letters) in the current hand.
    
    hand: dictionary (string int)
    returns: integer
    """
    count=0
    for key in hand.keys():
        count+=hand[key]
    return count

In [7]:
def playHand(hand, wordList, n):
    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    * The user may input a word or a single period (the string ".") 
      to indicate they're done playing
    * Invalid words are rejected, and a message is displayed asking
      the user to choose another word until they enter a valid word or "."
    * When a valid word is entered, it uses up letters from the hand.
    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.
    * The sum of the word scores is displayed when the hand finishes.
    * The hand finishes when there are no more unused letters or the user
      inputs a "."

      hand: dictionary (string -> int)
      wordList: list of lowercase strings
      n: integer (HAND_SIZE; i.e., hand size required for additional points)
      
    """
    total=0
    score=0
    word=''
    while calculateHandlen(hand)>0:
        print ("Current Hand:")
        displayHand(hand)
        word=input('Enter word, or a '+'"."'+' to indicate that you are finished:')
        if word=='.':
            print (' Goodbye! Total score:'+str(total)+' points. ')
            break
        else:
            if isValidWord(word,hand,wordList)==True:
                score=getWordScore(word,n)
                total+=score
                print  ('"'+word+'"'+' scored '+str(score)+' points.')
                print ("Total: "+ str(total)+" points")
                hand=updateHand(hand,word)
            else:
                print (' Invalid word, please try again.')
    if calculateHandlen(hand)==0:
        print ('Run out of letters. Total score:'+str(total)+' points.')
    return None



In [8]:
def playGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.
 
    1) Asks the user to input 'n' or 'r' or 'e'.
      * If the user inputs 'n', let the user play a new (random) hand.
      * If the user inputs 'r', let the user play the last hand again.
      * If the user inputs 'e', exit the game.
      * If the user inputs anything else, tell them their input was invalid.
 
    2) When done playing the hand, repeat from step 1
    """
    hand=None 
    ans=''
    while ans!='e':
        ans=input('Enter n to deal a new hand, r to replay the last hand, or e to end game: ')
        if ans=='n':
            hand=dealHand(HAND_SIZE)
            playHand(hand,wordList,HAND_SIZE)
        elif ans=='r':
            if hand==None:
                print ('You have not played a hand yet.') 
                print ('Please play a new hand first!')
               
            else:
                playHand(hand,wordList,HAND_SIZE)
        elif ans!='n' and ans!='e' and ans!='r':
            print ('Invalid command.')
    return None
            

In [9]:
def compChooseWord(hand, wordList, n):
    """
    Given a hand and a wordList, find the word that gives 
    the maximum value score, and return it.

    This word should be calculated by considering all the words
    in the wordList.

    If no words in the wordList can be made from the hand, return None.

    hand: dictionary (string -> int)
    wordList: list (string)
    returns: string or None
    """
    word=None
    score=0
    for i in range(len(wordList)):
        if isValidWord(wordList[i], hand, wordList)==True:
            a=getWordScore(wordList[i],n)
            if score<a:
                score=a
                word=wordList[i]
    if word==None:
        return None
    else:
        return word
    
   

In [10]:
def compPlayHand(hand, wordList, n):
    """
    Allows the computer to play the given hand, following the same procedure
    as playHand, except instead of the user choosing a word, the computer 
    chooses it.

    1) The hand is displayed.
    2) The computer chooses a word.
    3) After every valid word: the word and the score for that word is 
    displayed, the remaining letters in the hand are displayed, and the 
    computer chooses another word.
    4)  The sum of the word scores is displayed when the hand finishes.
    5)  The hand finishes when the computer has exhausted its possible
    choices (i.e. compChooseWord returns None).
 
    hand: dictionary (string -> int)
    wordList: list (string)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    """
    total=0
    score=0
    word=''
    while word!=None and calculateHandlen(hand)>0:
        print ("Current Hand:")
        displayHand(hand)
        word=compChooseWord(hand,wordList,n)
        if word==None:
            break
        else:
            score=getWordScore(word,n)
            total+=score
            print  ('"'+word+'"'+' earned '+str(score)+' points.')
            print ("Total: "+ str(total)+" points")
            hand=updateHand(hand,word)
           
    print ('Total score:'+str(total)+' points. ')
    return None


In [11]:
def compChooseWord(hand,wordList,n):
    word=None
    score=0
    for i in range(len(wordList)):
        if isValidWord(wordList[i], hand, wordList):
            a=getWordScore(wordList[i],n)
            if score<a:
                score=a
                word=wordList[i]
    if word==None:
        return None
    else:
        return word

In [12]:
def compPlayGame(wordList):
    """
    Allow the user to play an arbitrary number of hands.
 
    1) Asks the user to input 'n' or 'r' or 'e'.
        * If the user inputs 'e', immediately exit the game.
        * If the user inputs anything that's not 'n', 'r', or 'e', keep asking them again.

    2) Asks the user to input a 'u' or a 'c'.
        * If the user inputs anything that's not 'c' or 'u', keep asking them again.

    3) Switch functionality based on the above choices:
        * If the user inputted 'n', play a new (random) hand.
        * Else, if the user inputted 'r', play the last hand again.
          But if no hand was played, output "You have not played a hand yet. 
          Please play a new hand first!"
        
        * If the user inputted 'u', let the user play the game
          with the selected hand, using playHand.
        * If the user inputted 'c', let the computer play the 
          game with the selected hand, using compPlayHand.

    4) After the computer or user has played the hand, repeat from step 1

    wordList: list (string)
    """
    hand=None 
    ans=''
    play=''
    while ans!='e':
        ans=input('Enter n to deal a new hand, r to replay the last hand, or e to end game: ')
        if ans=='n':
            hand=dealHand(HAND_SIZE)
            play=''
            while play!='u' and play!='c':
                play=input("Enter u to have yourself play, c to have the computer play: ")
                if play=='u':
                    playHand(hand,wordList,HAND_SIZE)
                elif play=='c':
                    compPlayHand(hand,wordList,HAND_SIZE)
                else:
                    print ("Invalid Command")
        elif ans=='r':
            if hand==None:
                print ('You have not played a hand yet.')
                print ('Please play a new hand first!')
               
            else:
                play=input("Enter u to have yourself play, c to have the computer play: ")
                if play=='u':
                    playHand(hand,wordList,HAND_SIZE)
                if play=='c':
                    compPlayHand(hand,wordList,HAND_SIZE)
        elif ans!='n' and ans!='e' and ans!='r':
            print ('Invalid command.')
    return None



In [13]:
def displayHand(hand):
    """
    Displays the letters currently in the hand.

    For example:
    >>> displayHand({'a':1, 'x':2, 'l':3, 'e':1})
    Should print out something like:
       a x x l l l e
    The order of the letters is unimportant.

    hand: dictionary (string -> int)
    """
    for letter in hand.keys():
        for j in range(hand[letter]):
             print (letter)              # print all on the same line
    print()                               # print an empty line



In [14]:
VOWELS = 'aeiou'
CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
HAND_SIZE = 7

SCRABBLE_LETTER_VALUES = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1, 's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10
}

WORDLIST_FILENAME = "words_wordgame.txt"

def loadWords():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print ("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # wordList: list of strings
    wordList = []
    for line in inFile:
        wordList.append(line.strip().lower())
    print ("  ", len(wordList), "words loaded.")
    return wordList



In [15]:
def getFrequencyDict(sequence):
    """
    Returns a dictionary where the keys are elements of the sequence
    and the values are integer counts, for the number of times that
    an element is repeated in the sequence.

    sequence: string or list
    return: dictionary
    """
    # freqs: dictionary (element_type -> int)
    freq = {}
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
    return freq

In [16]:
if __name__ == '__main__':
    wordList = loadWords()
    compPlayGame(wordList)


Loading word list from file...
   83667 words loaded.
Enter n to deal a new hand, r to replay the last hand, or e to end game: n
Enter u to have yourself play, c to have the computer play: u
Current Hand:
u
a
b
d
g
q
l

Enter word, or a "." to indicate that you are finished:glad
"glad" scored 24 points.
Total: 24 points
Current Hand:
u
b
q

Enter word, or a "." to indicate that you are finished:.
 Goodbye! Total score:24 points. 
Enter n to deal a new hand, r to replay the last hand, or e to end game: e
